In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad.
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/learning-datasets/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if (logs.get('accuracy') > DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!\n")
      self.model.stop_training = True

callbacks = myCallback()

--2023-08-30 14:21:06--  https://storage.googleapis.com/learning-datasets/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 64.233.181.128, 64.233.182.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.06s   

2023-08-30 14:21:06 (45.9 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [14]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [15]:
# This code block should create an instance of an ImageDataGenerator called train_datagen
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [16]:
# This code block should call model.fit and train for
# a number of epochs.
history = model.fit(
    train_generator,
    epochs=15,
    steps_per_epoch=8,
    verbose=1,
      callbacks=[callbacks])

# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
8/8 [==============================] - 5s 407ms/step - loss: 1.2700 - accuracy: 0.5875
Epoch 2/15
8/8 [==============================] - 2s 261ms/step - loss: 0.4396 - accuracy: 0.8125
Epoch 3/15
8/8 [==============================] - 2s 255ms/step - loss: 0.2460 - accuracy: 0.9125
Epoch 4/15
8/8 [==============================] - 4s 471ms/step - loss: 0.1979 - accuracy: 0.9375
Epoch 5/15
8/8 [==============================] - 2s 255ms/step - loss: 0.1888 - accuracy: 0.9000
Epoch 6/15
8/8 [==============================] - 2s 258ms/step - loss: 0.0771 - accuracy: 0.9625
Epoch 7/15
8/8 [==============================] - 2s 250ms/step - loss: 0.1132 - accuracy: 0.9625
Epoch 8/15
8/8 [==============================] - 2s 290ms/step - loss: 0.1291 - accuracy: 0.9375
Epoch 9/15
8/8 [==============================] - 3s 351ms/step - loss: 0.0772 - accuracy: 0.9500
Epoch 10/15
8/8 [==============================] - 2s 242ms/step - loss: 0.0743 - accuracy: 0.9625
Epoch 11/15
8/8 [==